In [1]:
import gizmo_analysis as gizmo
import h5py
import halo_analysis as halo
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.interpolate as interp
import scipy.signal as signal
import utilities as ut
from gc_utils import get_descendants_halt, iteration_name, main_prog_halt, snapshot_name  # type: ignore


In [2]:
sim = "m12i"
sim_dir = "/Users/z5114326/Documents/simulations/"
fire_dir = sim_dir + sim + "/" + sim + "_res7100"

proc_file = sim_dir + sim + "/" + sim + "_processed.hdf5"
proc_data = h5py.File(proc_file, "r")  # open processed data file

snap_dir = fire_dir + "/snapshot_times.txt"
snap_data = pd.read_table(snap_dir, comment="#", header=None, sep=r"\s+")
snap_data.columns = [
    "index",
    "scale_factor",
    "redshift",
    "time_Gyr",
    "lookback_time_Gyr",
    "time_width_Myr",
]

snap_pub_dir = sim_dir + "/snapshot_times_public.txt"
snap_pub_data = pd.read_table(snap_pub_dir, comment="#", header=None, sep=r"\s+")
snap_pub_data.columns = [
    "index",
    "scale_factor",
    "redshift",
    "time_Gyr",
    "lookback_time_Gyr",
    "time_width_Myr",
]

In [3]:
snap = 77

hals = halo.io.IO.read_catalogs("index", [77, 600], fire_dir, assign_hosts_rotation=True)
# hals = halo.io.IO.read_catalogs("index", [77, 600], fire_dir)


# in utilities.simulation.Snapshot():
* reading:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/snapshot_times.txt

* using snapshot indices = [ 77 600]
  redshifts = [4.47826052 0.        ]


# in halo_analysis.halo_io.IO():
* read 9832 halos from:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/halo/rockstar_dm/catalog_hdf5/halo_077.hdf5

# in halo_analysis.halo_io.Particle():
* read 9832 halos, 301 have star particles, from:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/halo/rockstar_dm/catalog_hdf5/star_077.hdf5
* assigning primary host and coordinates wrt it to halo catalog...  finished


# in gizmo_analysis.gizmo_track.ParticleCoordinate():
  read 1 host (position, velocity, principal axes) from:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/track/host_coordinates.hdf5
  host1 position = (43057.26, 42130.64, 44311.36) [kpc comoving]
  host1 velocity = (-11.0, 39.1, 70.4) [km/s]
  host1 axis ratios = (0.53, 0.65, 0.81)

* read 35838 halos from

In [4]:
part = gizmo.io.Read.read_snapshots(
    ["dark", "star"], "index", snap, simulation_directory=fire_dir, assign_hosts_rotation=True
)

# part = gizmo.io.Read.read_snapshots(["dark", "star"], "index", snap, simulation_directory=fire_dir)


# in utilities.simulation.Snapshot():
* reading:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/snapshot_times.txt

  using snapshot index = 77, redshift = 4.478


# in gizmo_analysis.gizmo_io.Read():
* reading header from:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/output/snapdir_077/snapshot_077.0.hdf5
  snapshot contains the following number of particles:
    dark      (id = 1): 70514272 particles
    dark2     (id = 2): 5513331 particles
    gas       (id = 0): 70388512 particles
    star      (id = 4): 125821 particles
    blackhole (id = 5): 0 particles

* reading the following
  species: ['dark', 'star']

* reading particles from:
    snapshot_077.0.hdf5
    snapshot_077.1.hdf5
    snapshot_077.2.hdf5
    snapshot_077.3.hdf5

* reading cosmological parameters from:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/initial_condition/ic_agora_m12i.conf

* checking sanity of particle properties


# in gizmo_analysis.gizmo_track.ParticleCoordinate():
  r

From part 77

host1 position = (43057.26, 42130.64, 44311.36) [kpc comoving]

In [5]:
host_idx = np.where(hals[snap].prop("host.distance.principal.total") == 0)[0][0]
hals[snap]["position"][host_idx]

array([43056.184, 42130.63 , 44310.984], dtype=float32)

In [6]:
host_idx = np.where(hals[snap].prop("host.distance.principle.total") == 0)[0][0]
hals[snap]["position"][host_idx]

array([43056.184, 42130.63 , 44310.984], dtype=float32)

In [7]:
dist_vect = ut.particle.get_distances_wrt_center(
    part,
    species=["dark", "star"],
    center_position=hals[snap]["position"][host_idx],
    rotation=None,
)

In [8]:
dist_vect["dark"][0]

array([ 57.23848718, -33.88696372,   9.04401168])

In [9]:
part["dark"]["position"][0] - hals[snap]["position"][host_idx]

array([ 313.56733691, -185.641611  ,   49.54545094])

In [10]:
ut.coordinate.get_distances(
    part["dark"]["position"][0],
    hals[snap]["position"][host_idx],
    part.info["box.length"],
    part.snapshot["scalefactor"],
    total_distance=False,
)  # [kpc physical]

array([ 57.23848718, -33.88696372,   9.04401168])

In [11]:
ut.coordinate.get_distances(
    part["dark"]["position"][0],
    hals[snap]["position"][host_idx],
    hals[snap].info["box.length"],
    hals[snap].snapshot["scalefactor"],
    # total_distance,
)  # [kpc physical]

array([ 57.23858168, -33.88701967,   9.04402661])

In [12]:
part["dark"].prop("host.distance.principal", 0)

array([ 50.63068622, -21.79208763, -38.00478108])

In [13]:
part["dark"].prop("host.distance.principle", 0)

array([ 57.04168656, -33.88838981,   8.97484625])

In [14]:
np.linalg.norm(part["dark"].prop("host.distance.principal", 0))

66.95315418539779

In [15]:
np.linalg.norm(part["dark"].prop("host.distance.principle", 0))

66.95315403655763

In [16]:
# lets do it with galaxy centres

ut.coordinate.get_distances(
    hals[snap]["position"][10],
    hals[snap]["position"][host_idx],
    hals[snap].info["box.length"],
    hals[snap].snapshot["scalefactor"],
    # total_distance,
)  # [kpc physical]

array([-201.62328 ,   81.791466,  -79.352844], dtype=float32)

In [17]:
hals[snap].prop("host.distance.principle", 10)

array([-201.62328 ,   81.791466,  -79.352844], dtype=float32)

In [18]:
hals[snap].prop("host.distance.principal", 10)

array([-193.54637,   99.27209,   79.52053], dtype=float32)

HOLY SHIT IT WORKS

In [19]:
vel_vect = ut.particle.get_velocities_wrt_center(
    part,
    species=["dark", "star"],
    center_position=hals[snap]["position"][host_idx],
    # center_velocity=hals[snap]["velocity"][host_idx],
    rotation=None,
)

In [20]:
vel_vect["dark"][0]

array([-85.9688   ,   2.9954295, -30.679682 ], dtype=float32)

In [21]:
part["dark"].prop("host.velocity.principal", 0)

array([-87.89803,  23.71265,   8.39786], dtype=float32)

In [22]:
part["dark"].prop("host.velocity.principle", 0)

array([-86.061935 ,   2.9947548, -30.712414 ], dtype=float32)

In [23]:
vel_vect = ut.particle.get_velocities_wrt_center(
    part,
    species=["dark", "star"],
    center_position=hals[snap]["position"][host_idx],
    center_velocity=hals[snap]["velocity"][host_idx],
    rotation=None,
)

vel_vect["dark"][0]

array([-118.61138 ,    6.050033,  -43.019585], dtype=float32)

In [24]:
ut.coordinate.get_velocity_differences(
    part["dark"]["velocity"][0],
    hals[snap]["velocity"][host_idx],
    part["dark"]["position"][0],
    hals[snap]["position"][host_idx],
    part.info["box.length"],
    part.snapshot["scalefactor"],
    part.snapshot["time.hubble"],
    total_velocity=False,
)

array([-118.61138 ,    6.050033,  -43.019585], dtype=float32)

In [25]:
ut.coordinate.get_velocity_differences(
    part["dark"]["velocity"][0],
    part.host["velocity"],
    part["dark"]["position"][0],
    part.host["position"],
    part.info["box.length"],
    part.snapshot["scalefactor"],
    part.snapshot["time.hubble"],
    total_velocity=False,
)

array([[-86.061935 ,   2.9947548, -30.712414 ]], dtype=float32)

IT LOOKS LIKE THERE IS A DIFFERENT IN HOST VELOCITY

In [26]:
np.linalg.norm(hals[snap]["velocity"][host_idx])

92.85362

In [27]:
np.linalg.norm(part.host["velocity"])

81.317474

WTF!!!!!!

In [28]:
hals[snap].host["velocity"]

array([[-10.982577,  39.134605,  70.43009 ]], dtype=float32)

In [29]:
hals[snap]["velocity"][host_idx]

array([21.66, 36.08, 82.77], dtype=float32)

In [30]:
part.host["velocity"]

array([[-10.982577,  39.134605,  70.43009 ]], dtype=float32)

In [31]:
hals[snap].host["position"]

array([[43057.26 , 42130.637, 44311.363]], dtype=float32)

In [32]:
hals[snap]["position"][host_idx]

array([43056.184, 42130.63 , 44310.984], dtype=float32)

In [33]:
part.host["position"]

array([[43057.26 , 42130.637, 44311.363]], dtype=float32)

In [34]:
print(hals[snap].host["velocity"] - hals[snap]["velocity"][host_idx])

[[-32.64258     3.0546036 -12.339905 ]]


In [35]:
ut.particle.get_center_velocities_or_accelerations(
    part,
    property_kind="velocity",
    species_name="star",
    part_indicess=None,
    weight_property="mass",
    distance_max=10,
    center_positions=None,
    return_single_array=True,
    verbose=True,
)


# in utilities.particle.get_center_velocities_or_accelerations():
* assigning velocity for 1 center/host, weighting star particles by mass
  host1 velocity = (-10.6, 39.8, 69.4) [km/s]


array([-10.581415,  39.823235,  69.449585], dtype=float32)

## LETS DO A TEST TO JUST CONFIRM THE PROCESS

In [36]:
host_idx = np.where(hals[snap].prop("host.distance.principal.total") == 0)[0][0]
hals[snap]["position"][host_idx]

array([43056.184, 42130.63 , 44310.984], dtype=float32)

In [37]:
# ut.particle.get_center_positions(
#     part,
#     species_name="star",
#     part_indicess=None,
#     weight_property="mass",
#     center_number=1,
#     exclusion_distance=400,
#     center_positions=None,
#     distance_max=np.inf,
#     return_single_array=True,
#     verbose=True,
# )

In [51]:
# We will find the distances to part["dark"] index 0

ut.coordinate.get_distances(
    part["dark"]["position"][0],
    hals[snap]["position"][host_idx],
    part.info["box.length"],
    part.snapshot["scalefactor"],
    total_distance=False,
)  # [kpc physical]

array([ 57.23848718, -33.88696372,   9.04401168])

In [96]:
part["dark"].prop("host.distance.principal", 0)

array([ 50.63068622, -21.79208763, -38.00478108])

In [97]:
part["dark"].prop("host.distance", 0)

array([ 57.04168656, -33.88838981,   8.97484625])

In [ ]:
# compute un-rotated coordinates wrt the host
unrotate_tensor = part.host["rotation"][0].transpose()
ut.coordinate.get_coordinates_rotated(part["dark"].prop("host.distance.principal", 0), unrotate_tensor)

array([ 57.04168667, -33.8883899 ,   8.9748475 ])

In [57]:
ut.coordinate.get_coordinates_rotated(part["dark"].prop("host.velocity.principal", 0), unrotate_tensor)

array([-86.061935 ,   2.9947553, -30.712418 ], dtype=float32)

# OKAY WE GOT IT FINN -> LETS PUT IT ALL TOGETHER

In [58]:
ut.particle.get_center_velocities_or_accelerations(
    part,
    property_kind="velocity",
    species_name="star",
    part_indicess=None,
    weight_property="mass",
    distance_max=10,
    center_positions=None,
    return_single_array=True,
    verbose=True,
)


# in utilities.particle.get_center_velocities_or_accelerations():
* assigning velocity for 1 center/host, weighting star particles by mass
  host1 velocity = (-10.6, 39.8, 69.4) [km/s]


array([-10.581415,  39.823235,  69.449585], dtype=float32)

In [61]:
part.host["velocity"], hals[snap].host["velocity"]

(array([[-10.982577,  39.134605,  70.43009 ]], dtype=float32),
 array([[-10.982577,  39.134605,  70.43009 ]], dtype=float32))

In [72]:
# 'velocity' : 3-D velocity, along simulation's (cartesian) x,y,z grid [km/s]

hals[snap]["velocity"][host_idx]

array([21.66, 36.08, 82.77], dtype=float32)

In [95]:
hals[snap]["velocity"][host_idx]

array([21.66, 36.08, 82.77], dtype=float32)

In [98]:
# ut.coordinate.get_velocities_in_coordinate_system(values, distance_vectors, "cartesian", coordinate_system)

In [86]:
# ut.coordinate.get_center_velocity(
#     velocities,
#     weights=None,
#     positions=None,
#     center_position=None,
#     distance_max=20,
#     periodic_length=None,
# )

In [73]:
hals[snap]["host.velocity"][host_idx]

array([0., 0., 0.], dtype=float32)

# Lets do this for real

In [125]:
# second largest galaxy
galaxy_b_idx = np.where(hals[snap]["mass"] == np.flip(np.sort(hals[snap]["mass"]))[1])[0][0]

# dark matter close to the centre of galaxy_B
particle_idx = 31986394

In [134]:
# this is what I need to match
hals[snap].prop("host.distance.principal", galaxy_b_idx)

array([ 100.06183, -256.75357, -249.67384], dtype=float32)

In [ ]:
rotate_tensor = hals[snap].host["rotation"][0]
unrotate_tensor = rotate_tensor.transpose()

array([ 143.40086, -321.53067,  119.69134], dtype=float32)

In [140]:
ut.coordinate.get_coordinates_rotated(
    hals[snap].prop("host.distance.principal", galaxy_b_idx), unrotate_tensor
)

array([ 143.40086, -321.53067,  119.69134], dtype=float32)

In [137]:
# position of galaxy B in reference to galaxy A

galaxy_b_pos_a_unrotated = ut.coordinate.get_distances(
    hals[snap]["position"][galaxy_b_idx],
    hals[snap]["position"][host_idx],
    part.info["box.length"],
    part.snapshot["scalefactor"],
    total_distance=False,
)  # [kpc physical]

galaxy_b_pos_a_unrotated

array([ 143.40062, -321.53015,  119.69114], dtype=float32)

In [139]:
galaxy_b_pos_a_rotated = ut.coordinate.get_coordinates_rotated(galaxy_b_pos_a_unrotated, rotate_tensor)
galaxy_b_pos_a_rotated

array([ 100.06166, -256.75317, -249.67345], dtype=float32)

In [142]:
# now lets do the same for the particle

# this is what I need to match
part["dark"].prop("host.distance.principal", particle_idx)

array([  99.87586756, -256.63107867, -249.68264204])

In [143]:
ut.coordinate.get_coordinates_rotated(
    part["dark"].prop("host.distance.principal", particle_idx), unrotate_tensor
)

array([ 143.22875382, -321.51075297,  119.5511743 ])

In [144]:
# position of galaxy B in reference to galaxy A

particle_pos_a_unrotated = ut.coordinate.get_distances(
    part["dark"]["position"][particle_idx],
    hals[snap]["position"][host_idx],
    part.info["box.length"],
    part.snapshot["scalefactor"],
    total_distance=False,
)  # [kpc physical]

particle_pos_a_unrotated

array([ 143.42555476, -321.50932318,  119.62033025])

In [145]:
particle_pos_a_rotated = ut.coordinate.get_coordinates_rotated(particle_pos_a_unrotated, rotate_tensor)
particle_pos_a_rotated

array([ 100.07819675, -256.68025201, -249.69534056])

In [154]:
# now lets get the distance between galaxy a and star particle

test_rotate = ut.coordinate.get_distances(
    particle_pos_a_rotated,
    galaxy_b_pos_a_rotated,
    part.info["box.length"],
    part.snapshot["scalefactor"],
    total_distance=False,
)  # [kpc physical]

test_rotate, np.linalg.norm(test_rotate)

(array([ 0.00301847,  0.01331113, -0.00399651]), 0.014222143123631481)

In [156]:
test_unrotate = ut.coordinate.get_distances(
    particle_pos_a_unrotated,
    galaxy_b_pos_a_unrotated,
    part.info["box.length"],
    part.snapshot["scalefactor"],
    total_distance=False,
)  # [kpc physical]

test_unrotate, np.linalg.norm(test_unrotate)

(array([ 0.00455167,  0.00380197, -0.01292545]), 0.014221109526467983)

DONT FORGET WILL NEED TO DO VELOCITY DISPERSION